In [ ]:
#konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 460kB 38.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import files
from konlpy.tag import Komoran
import pandas as pd
import re
from tqdm import tqdm

tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
comments = pd.read_pickle('/content/commentList.pickle')
comments.head()

,title,author,comment,like
0,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,피식대학Psick Univ,B대면데이트 첫번째 소개팅남<br />방재호 instagram@BJH_Dreamlife,592
1,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,으앙,처음만났는데 대체 제얘기는 언제해요.... ㅜㅜ,0
2,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,Do it,광용샘....,0
3,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,말랑말랑,저 이마의 끝이 어딜까...,1
4,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,진성,근데 피부는 진짜 나쁘지 않ㄴ네,0


In [ ]:
comments_character = comments.copy()
comments_character['character'] = comments['title'].str.findall(re.compile('트 ([가-힣]+?)/')).str[0]

In [ ]:
comments_character.to_pickle('comments_character.pickle')
comments_character.to_csv('comments_character.csv', encoding='utf-8-sig')

In [ ]:
comments_character = pd.read_pickle('/content/comments_character.pickle')

In [ ]:
comments_character['comment'][15:20]

15                      귀여워요 방!재!호♡
16    ❤️❤️당신을 사랑하는 !🥺 🥵💕너무 뜨거운 🤢😻 🥵
17     눈썹때매 집중이안되요.... 눈썹그려주고 싶다...
18        \  _  / 눈이 이렇게 올라가있어.....
19                   연기력 미쳤다 진짜...♡
Name: comment, dtype: object

In [ ]:
tokenizer = Komoran() # global variable

def text_cleaning(doc):
  text = doc.replace('<br />', ' ')
  text = re.sub('&.+?;', ' ', text) # &quot; / &lt; / &gt; / &#39;
  text = re.sub('<a.+?/a>', '', text)
  text = re.sub('[^가-힣0-9a-zA-Z,\.\s]', '', string = text).strip()
  return text

# 감탄사(IC), 형용사(VA), 일반명사(NNG), 고유명사(NNP), 외국어(SL)

def text_preprocess(doc):
  # 텍스트 전처리
  clean_doc =  text_cleaning(doc)
  
  # 품사 붙이기
  token_tagged = tokenizer.pos(clean_doc)

  # 추출
  words = [s for s, t in token_tagged if t in ['IC', 'VA', 'SL', 'NNG', 'NNP'] and len(s) > 1]

  # 한 문장으로 합치기 
  preprocessed = ' '.join(words)
  return preprocessed

def download(df_file):
  file_name = 'comments_prep_full'
  df_file.to_pickle(file_name + '.pickle')
  df_file.to_csv(file_name + '.csv', encoding='utf-8-sig')
  files.download(file_name + '.pickle')
  files.download(file_name + '.csv')

In [ ]:
comments_character['prep'] = comments_character['comment'].progress_apply(lambda x: text_preprocess(x))

100%|██████████| 64400/64400 [00:37<00:00, 1724.35it/s]


In [ ]:
comments_character[15:20]

,title,author,comment,like,character,prep
15,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,사경주,귀여워요 방!재!호♡,0,방재호,귀엽 방재
16,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,신효섭꿀성대,❤️❤️당신을 사랑하는 !🥺 🥵💕너무 뜨거운 🤢😻 🥵,0,방재호,사랑 뜨겁
17,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,배주안,눈썹때매 집중이안되요.... 눈썹그려주고 싶다...,0,방재호,눈썹 집중 이안 눈썹
18,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,소금sogum,\ _ / 눈이 이렇게 올라가있어.....,1,방재호,이렇
19,[B대면데이트]#1. 첫번째 데이트 방재호/33/다단계,sk lee,연기력 미쳤다 진짜...♡,0,방재호,연기력 진짜


In [ ]:
download(comments_character)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>